In [75]:
from dotenv import load_dotenv
load_dotenv()

True

In [76]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages
prompt_template = ChatPromptTemplate([
    ('system', """You are a professional chatbot working at a horse racing track, providing detailed, helpful, and dynamic information based on the following sources:
    Quick Information Access: Summarize key details such as race schedules, race types, and betting methods in an easy-to-understand way for beginners.
    "Please provide the information based on the following websites:

https://namu.wiki/w/%EA%B2%BD%EB%A7%88/%ED%95%9C%EA%B5%AD
https://race.kra.co.kr/raceguide/RaceProcessGetService.do
https://knetz.kra.co.kr/index.do
https://park.kra.co.kr/seoul_main.do?strToken=XXXXX
http://www.okrace.co.kr/okrace/ai_info.php"
    Dynamic Betting Guidance: Provide tips for different types of bets (Win, Place, Exacta, etc.) and display real-time odds.
    Location-Specific Information: Offer details about the specific racetrack, including facilities, events, and location-based info.
    Track-Specific Information: Customize responses based on unique features and events of each racetrack.
    Ensure that all responses are friendly, accurate, and cater to both newcomers and regular visitors."""),
    
    ('user', '{question}')
])

# 출력 파서 생성
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

SyntaxError: invalid syntax (2568194066.py, line 2)

## pdf 파일 로드

In [77]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_community.document_loaders import PyPDFLoader
import os

# 여러 PDF 파일 경로를 리스트로 생성
pdf_folder = 'C:/Workspaces/dl_nlp/Choisoovin/10_langchain/04_ai_Sommelier_rag/pdfs'
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

# 각 PDF 파일을 로드하여 문서로 처리
pdf_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents.extend(loader.load())  # 문서를 리스트에 추가

print(f"Loaded {len(pdf_documents)} documents.")

Loaded 203 documents.


## CSV 파일 로드

import os
from langchain_community.document_loaders import CSVLoader

# CSV 저장된 폴더 경로 지정
csv_folder = 'C:/Workspaces/dl_nlp/Choisoovin/10_langchain/04_ai_Sommelier_rag/csvs'

# 폴더 내 모든 CSV 파일 경로 리스트 생성
csv_files = [os.path.join(csv_folder, file) for file in os.listdir(csv_folder) if file.endswith('.csv')]

# 폴더 내 모든 CSV 파일을 Document 객체로 로드
csv_documents = []
for csv_file in csv_files:
    loader = CSVLoader(file_path=csv_file)  
    csv_documents.extend(loader.load())    


## web 방식

from langchain.document_loaders import WebBaseLoader

# 조회하고 싶은 URL들의 리스트
urls = [
    "http://www.okrace.co.kr/okrace_guide/race_guide1.php",
    "http://www.okrace.co.kr/okrace_guide/race_guide2.php",
    "http://www.okrace.co.kr/okrace_guide/race_guide9.php"
]

# WebBaseLoader에 리스트 전달
loader = WebBaseLoader(urls)

# 로드된 문서 가져오기
web_documents = loader.load()

# 출력 확인
len(web_documents)

web_documents[1]

## 데이터 병합

In [81]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
docs = [document.page_content for document in pdf_documents]

all_documents = pdf_documents #+ web_documents

## Vector Store 구축 - FAISS 사용

In [87]:
from langchain.vectorstores import FAISS

vectors = FAISS.from_documents(all_documents, embeddings) #documents : Document
vectors

## Retriever 객체 생성

In [88]:
retriever = vectors.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023DF9AD6540>, search_kwargs={})

In [91]:
# 벡터스토어 검색 및 LLM 질의

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

#질의 객체
rtvr_qa=RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type = 'stuff' #검색된 내용을 하나의 텍스트로 생성해서 LLM에 전달
)

#llm 질의
response1 = rtvr_qa.invoke('auto bet에 대해 알려줘')
print(response1)

{'query': 'auto bet에 대해 알려줘', 'result': '죄송하지만, 제공된 정보에는 "auto bet"에 대한 구체적인 내용이 포함되어 있지 않습니다. "auto bet"에 대한 정보를 원하신다면, 관련된 다른 자료나 웹사이트를 참고하시기 바랍니다.'}


In [ ]:
#response1 = rtvr_qa.invoke('오토벳에 대해 알려줘')
#print(response1)
#TODO pdf 업로드 시, 내용이 누락될 가능성?
#race_guide.pdf에 오토벳에 대한 내용이 있는데 뱉어주는 게 없다.
#민하님께서 guide.pdf만 넣고 돌렸을 땐 관련 내용이 잘 나왔는데, 여러 pdf 파일을 넣고 돌리니깐 해당 내용을 제공할 수 없단 답변을 받았다고 하심.
#이에 다 같이 돌려봤으나 똑같은 증상을 겪었다.